In [5]:
import pandas as pd
import numpy as np

#read all data
users = pd.read_json('data/users.jsonl', lines=True)
products = pd.read_json('data/products.jsonl', lines=True)
sessions = pd.read_json('data/sessions.jsonl', lines=True)
deliveries = pd.read_json('data/deliveries.jsonl', lines=True)

ready_data = pd.read_csv('data/parsed_data.csv')

# Projekt IUM
## Etap 1
###### Marcel Kawski, Konrad Kulesza

# Opis zadania
---
### Definicja problemu biznesowego

„Niektórzy klienci nie mogą zdecydować się na zakup oglądając produkt na stronie –
ale pewnie, gdybyśmy wiedzieli, które to są osoby, to odpowiednia zniżka skłoniłaby
ich do zakupu”.

Wspomaganie sklepu internetowego w przyznawaniu zniżek w celu zwiększenia liczby sprzedanych produktów


### Biznesowe kryterium sukcesu
Na ten moment odsetek udanych sesji wynosi około 23%. Sukcesem będzie zwiększenie go o minimum 5 punktów procentowych.


### Kontekst naszego modelu
Nasz model będzie jednym komponentem całego, złożonego systemu. Wiemy, że inny zespół/model na podstawie m.in. naszych predykcji będzie próbował przyznawać takie wartości zniżek, aby klienci zostawiali w naszej firmie jak najwięcej pieniędzy. 

### Zadanie modelowania
Binarna klasyfikacja czy sesja danej osoby zakończy się zakupem(zakładamy, że prawdopodobieństwo >65% oznacza sukces)

### Analityczne kryterium sukcesu
Naiwny klasyfikator stwierdzający, że nikt nie kupi będzie miał skuteczność ~77%.

p = (liczba sesji poprawnie sklasyfikowanych / liczba sesji)

Sukcesem będzie osiągnięcie:
**p >= 82%**


-----
-----
-----



# Analiza danych
---
### Czyszczenie oraz obróbka dostępnych danych 

#### users.jsonl
Dane są poprawne.
Wykorzystamy: 
- `user_id`,
- `city`
- z `name` wydobędziemy płeć użytkownika(imie kończy się na `a` -> kobieta)

#### products.jsonl
Atrybuty wymagające interwencji:
- `price` - niektóre wartości ujemne albo bardzo wysokie. Ze względu na duży zbiór danych postanowiliśmy 'wyrzucić' dane rekordy. Jednak, w przypadku stwierdzenia, że mamy za mało danych to przywrócimy dane z ujemnymi wartościami i zwyczajnie zanegujemy ich wartość
- `category_path` - rozbiliśmy ścieżkę na węzły i zakodowaliśmy ją za pomocą `one-hot encoding`. Dzięki temu nie straciliśmy informacji o podobieństwie danych produktów.

Dodatkowo wykorzystamy:
- `rating`
- `price`

#### sessions.jsonl
Wymagają interwencji:
- `user_id` czasami jest nullem.
- `product_id` czasami jest nullem.
- odrzucamy eventy `VIEW_PRODUCT`, które trwają poniżej 2 sekund, uznajemy je za przypadkowe kliknięcie

Potraktowaliśmy braki jako `MCAR` i postanowiliśmy odrzucić błędne rekordy ze wzgledu na dużą liczebność zbioru. Jednak, jeżeli stwierdzimy, że mamy za mało danych uczących to wykorzystamy te rekordy wprowadzając atrybuty wskaźnikowe.

Dodatkowo widzimy, że dotychczas przyznawanymi zniżkami były wartości: {0, 5, 10, 15, 20}


#### deliveris.jsonl - nieużywamy

### Podsumowanie dostępnych danych
Po "konsultacji z klientem" otrzymaliśmy dodatkowe informacje o ocenach poszczególnych produktów oraz dowiedzieliśmy się, że metadane klientów nie są dostępne. 



## Dane wejściowe/wyjściowe - wnioski
Ze względu na brak większej liczby metadanych użytkowników, zmuszeni jesteśmy przewidywać sukces tylko na podstawie 'podobieństwa' innych sesji, dotychczasowej historii danego użytkownika, zniżek oraz informacji o produktach.

Danymi wejściowymi będzie wiersz opisujący sesję. Z dostępnych plików udało nam się początkowo otrzymać rekordy z następującymi atrybutami:
- `długość sesji`
- `średnia kwot przeglądanych produktów`
- `mediana kwot przeglądanych produktów`
- `kategorie przeglądanych produktów`
- `miasto użytkownika`
- `płeć użytkownika`
- `zaoferowaną zniżkę`
- `liczbę historycznych sesji danego użytkownika`
- `liczbę historycznych sesji danego użytkownika zakończonych sukcesem`
- `średnia ocena przeglądanych produktów`
- `minimalna --||--`
- `maksymalna --||--`


Wyjściem modelu będzie prawddopodobieństwo tego, że sesja zakończy się sukcesem.

Funkcją celu będzie zaproponowana na wykładzie entropia krzyżowa


----



In [6]:
ready_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9598 entries, 0 to 9597
Data columns (total 42 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             9598 non-null   int64  
 1   length                                 9598 non-null   object 
 2   discount                               9598 non-null   int64  
 3   user_id                                9598 non-null   int64  
 4   successful                             9598 non-null   bool   
 5   mean_price                             9598 non-null   float64
 6   median_of_price                        9598 non-null   float64
 7   min_rating                             9598 non-null   float64
 8   max_rating                             9598 non-null   float64
 9   mean_rating                            9598 non-null   float64
 10  sessions_so_far                        9598 non-null   int64  
 11  succ

Powyżej przedstawiamy dane wejśiowe, które analizowaliśmy.
Po analizie tych danych na pewno obetniemy(znaczącą) część atrybutów, ponieważ nie chcemy, aby nasz model uczył się na podstawie danych zaszumionych nieinformatywn.

Wnioski wynikające z analizy danych:
* Najbardziej informatywnym(pojedynczym) atrybutem wejściowym jest rzeczywiście przyznana zniżka
* Liczność każdej ze zniżek jest na bardzo podobnym pozimoie, najbardziej prawdopodobnym rozkładem jest rozkład jednostajny => nie musimy stosować żadnej ze strategii balansowania względem tego atrybutu
* Zdecydowana większość sesji zakończonych zakupem dotyczy klientów, którzy nigdy wcześniej nie kupowali nic w naszym sklepie. Oznacza to, że nie powinniśmy koncentrować się na historii zakupów danych klientów.
* Klienci przeglądają oraz kupiują tylko i wyłącznie produkty z tej samej kategorii.
* Wstępnie ustaliliśmy, że kategorie zakodowane za pomocą kodowania one-hot nie niosą żadnej informacji. 
* Na diagramie korelacji niestety nie można zauważyć żadnych silnych relacji pomiędzy określeniem sesji jako zakończoną sukcesem a innymi pojedynczymi atrybutami poza wielkością zniżki. Prawdopodobnie wyjście jest zależne od wielu zależnych od siebie czynników. Możemy zauważyć jedynie zależność pomiędzy cenami i ocenami kleintów w zależności od kategorii produktów, lecz nie jest to użyteczna z perspektywy naszego zadania informacja.
* Rozkłady udanych zakupów także nie różnią się ze względu na płeć.
